In [8]:
import os
from pathlib import Path

from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

from datasets import load_dataset
import cassio
from PyPDF2 import PdfReader
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
OPENAI_API_KEY_VAR = os.getenv('OPENAI_API_KEY')
DB_TOKEN_API_KEY = os.getenv('DB_TOKEN_API_KEY')
DB_ID_API_KEY = os.getenv('DB_ID_API_KEY')

In [13]:
pdf_reader = PdfReader("/media/kornellewy/jan_dysk_3/nlp_projects/datasets/budget_speech.pdf")

In [14]:
from typing_extensions import Concatenate

raw_text = ''
for i, page in enumerate(pdf_reader.pages):
    content = page.extract_text()
    if content:
        raw_text += content
    

In [16]:
cassio.init(token=DB_TOKEN_API_KEY, database_id=DB_ID_API_KEY)

In [17]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY_VAR)
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY_VAR)

/media/kornellewy/jan_dysk_3/nlp_projects/nlp_venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/media/kornellewy/jan_dysk_3/nlp_projects/nlp_venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn

In [19]:
astra_vectro_store = Cassandra(
    embedding=embeddings,
    table_name='qa_mini_demo',
    session=None,
    keyspace=None,
)

In [21]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)
texts = text_splitter.split_text(raw_text)

In [24]:
# uplad data and aplly embbeding
astra_vectro_store.add_texts(texts[:50])
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vectro_store)

In [27]:
question1 = "What is current gdp?"
question2 = "How much the agriculture target will be increased to and what the focus will be?"

first_question = True

for question in [question1, question2]:
    answer = astra_vector_index.query(question, llm=llm).strip()

    for doc, score in astra_vectro_store.similarity_search_with_score(question, k=4):
        print(round(score, 4), doc.page_content[:100])
    print("########################")

0.8998 rural areas to women as sole or joint owners have enhanced 
their dignity.  
Exemplary  Track Record
0.8915 resolutions, as the country opens up immense possibilities and 
opportunities” . It is our ‘Kartavya
0.8896 Economic Management  
28. The multipronged economic management over the  past 
ten years has complem
0.8894 borrowings and the total expenditure are estimated at ` 30.80  
and 47.66  lakh crore respectively. 
########################
0.9089 procurement, value addition, and crop insurance.      
Dairy Development  
54. A comprehensive progr
0.9087 trading volume of ` 3 lakh crore.  
16. The sector is poised for inclusive, balanced, higher growth 
0.9069 Department for Fisheries realizing the importance of assisting 
fishermen. This has resulted in doub
0.9047 boosting farmers’ income will be stepped up. Pradhan Mantri 
Kisan  Sampada Yojana has benefitted 38
########################
